In [10]:
import tensorflow as tf

In [6]:
import pickle

with open("Resume_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("word_to_index.pkl", "rb") as f:    
    word_to_index = pickle.load(f)

In [2]:
import spacy
Model=spacy.load('en_core_web_sm')

In [8]:
def preprocess(text2):
    text2=text2.replace('\t','')
    text2=text2.replace('\n','')

    import re

    patterns = ['\d{11}','\d{4}-\d{7}', '[a-z0-9]*@gmail.com','linkedin.com[\/ a-z-0-9A-Z]*','Resume','resume','Resume Headline','github.com[\/ a-z-0-9:.A-Z]*','Pakistan','Lahore','india','India','INDIA']
    combined_pattern = '|'.join(patterns)
    pattern=re.compile(combined_pattern)

    text2 = pattern.sub('',text2)

    doc=Model(text2)

    tokens_list=[]
    para=''
    for token in doc:
        if(not(token.is_punct)):
            tokens_list.append(token.lemma_)
        para=' '.join(tokens_list)
    text2=para
    return text2

In [11]:
text1 = "HeadlineFlutter Developer | Cross platform App DeveloperSkill Set C++ JAVA JAVASCRIPT MySQL HTML CSS React.js Flutter Dart Database Management System DBMS Algorithms Data Structures Object orient Programming OOP prefer Job Type full Time Remote Employement Details Flutter Developer Tech Alchemy Ltd Jul 2021 Present full time 1 prototyped several new feature for an ongoing project.2 decrease rework by 22 for the implement features.3 contribute consistently to improve code quality among all the flutter project of the company.1 prototype several new feature for an ongoing project 2 decrease rework by 22 for the implement feature 3 contribute consistently to improve code quality among all the flutter project of the company skill Dart fluttersalary 29,000 Monthly Notice Period 2 month Graduation in Information Technology BTech Bachelor of Technology from Maharaja Agrasen Institute of Technology Delhi Passout Year 2021 course type full Time Percentage Grade 7.8 Scale 10 Grading System Profile Summary Prolific flutter developer with a passion for metric and beat former good yet passionate about build world class cross platform application with a zeal to learn and adapt new technology strong engineering professional as Bachelor of Technology(B.Tech major in Information Technology from Maharaja Agrasen Institute of Technology Delhi"
text2 = """
Company Description
Vertex Infinity is an innovative software development company that provides web and mobile application solutions for clients ranging from startups to large enterprises globally. Specializing in scratch-based development for large-scale systems, as well as AI services and expertise in large language models, Vertex Infinity offers bespoke solutions that ensure scalability and resilience in today's fast-paced digital landscape.
Role Description
This is a full-time hybrid role for a Frontend Developer located in Lahore, with flexibility for some remote work. The Frontend Developer will be responsible for designing and developing dynamic user interfaces and intuitive user experiences, as well as ensuring responsive web design, software development, and web development.
Qualifications
    Expertise in Flutter Dart Database Management System
    Expertise in C++ JAVA JAVASCRIPT MySQL HTML CSS React.js
    Expertise in Front-End Development and Responsive Web Design
    Experience with UI/UX design and implementation
    Experience with HTML, CSS, JavaScript, and related frameworks and libraries such as React, Angular, Vue, etc.
    Knowledge of Back-End Web Development is a plus
    Experience with Git and other version control systems
"""
text1=preprocess(text1)
text2=preprocess(text2)

def text_to_sequence(text, word_to_index):
    tokens=text.split()
    sequence=[word_to_index[token] for token in tokens if token in word_to_index]
    return sequence

sequence1=text_to_sequence(text1, word_to_index)
sequence2=text_to_sequence(text2, word_to_index)

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length=4
sequence1_padded=pad_sequences([sequence1], maxlen=max_sequence_length, padding='post', truncating='post')
sequence2_padded=pad_sequences([sequence2], maxlen=max_sequence_length, padding='post', truncating='post')


embedding_layer=model.layers[0]
embedding_model=tf.keras.models.Model(inputs=model.input, outputs=embedding_layer.output)

embedding1=embedding_model(sequence1_padded)
embedding2=embedding_model(sequence2_padded)

embedding1_flat=tf.reshape(embedding1, embedding1.shape[1:])
embedding2_flat=tf.reshape(embedding2, embedding2.shape[1:])

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim=cosine_similarity(embedding1_flat, embedding2_flat)[0,0]
similarity_scaled=((cosine_sim+1)/2)*100
print("Similarity between Resume and Requirements are", int(similarity_scaled),'%')

Similarity between Resume and Requirements are 62 %
